# Twython 시작하기

In [ ]:
from twython import Twython           #Search API
from twython import TwythonStreamer   #Streaming API

CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

Twython <- <b>Search API</b>에 쓰인다.<br>
TwythonStreamer <- <b>Streaming API</b>에 쓰인다. <br><br>
CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET ... 모두 미리 변수에 할당하였다.

## Search API

In [ ]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET)

# "data science"라는 단어가 들어가 있는 트윗 검색
for status in twitter.search(q='"data science"')["statuses"]:
    user = status["user"]["screen_name"].encode('utf-8')
    text = status["text"].encode('utf-8')
    print(user,":",text.decode("utf-8"))

## Streaming API 

In [ ]:
tweets = []

class MyStreamer(TwythonStreamer):
    """Streaming API를 어떻게 사용할지 명시하기 위해 직접 만든
    TwythonStreamer의 하위 클래스"""

    def on_success(self, data):
        """트위터에서 보내 온 데이터를 어떻게 처리할까?
    여기서는 트윗을 dict 형태로 유지할 것이다"""

        # 영어로 작성된 트윗만 원한다면
        if data['lang'] == 'en':
            tweets.append(data)

        # 트윗을 적당히 모았다면 멈춤 (여기서는 5개 이상을 모았으면 멈춘다)
        if len(tweets) >= 5:
            self.disconnect()

    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

In [ ]:
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                        ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
stream.statuses.filter(track='data')
print("가져온 트윗 개수 : ",len(tweets))

In [ ]:
tweets[0]

In [ ]:
tweets[0]['created_at']

In [ ]:
tweets[1]['created_at']

## Streaming API + Sqlite3

예시가 아닌 실제 프로젝트를 진행할 떄는 불러온 트윗을 인메모리(in-memory)에 올려서 사용하는 것을 추천하지 않는다.<br>
먼저 파일이나 <b>데이터베이스로 저장</b>을 하고 작업을 하도록 하자.

In [ ]:
import sqlite3

conn = sqlite3.connect('C:/Users/lion5/Desktop/Study_Portfolio/동아리및학회/GrowthHackers/data-science-from-scratch-master/9장준비/3기 대상/DS.db')
curs = conn.cursor()

In [ ]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

In [ ]:
from twython import TwythonStreamer,Twython

In [ ]:
tweets = []

class MyStreamer(TwythonStreamer):
    """Streaming API를 어떻게 사용할지 명시하기 위해 직접 만든
    TwythonStreamer의 하위 클래스"""

    def on_success(self, data):
        """트위터에서 보내 온 데이터를 어떻게 처리할까?
    여기서는 트윗을 dict 형태로 유지할 것이다"""

        # 영어로 작성된 트윗만 원한다면
        if data['lang'] == 'en':
            data = data['text'].replace('"','`')
            curs.execute('insert into twitter values ("'+data+'")')
            tweets.append("0")

        # 트윗을 적당히 모았다면 멈춤 (여기서는 30개 이상을 모았으면 멈춘다)
        if len(tweets) >= 50:
            self.disconnect()
            conn.commit()
            conn.close()
            
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()


In [ ]:
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                        ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
# 공개된 트윗 중 data라는 키워드를 포함하고 있는 트윗을 받아 오기 시작함
stream.statuses.filter(track='data')